## Lee TH., Seregina E.: "Combining Forecasts under Structural Breaks Using Graphical LASSO"

### This R notebook can be used to reproduce the values for competing methods (everything except RD-FGL) in Table 1 and Table 2
### for both empirical applications

#### (Please refer to Python notebook to reproduce RD-FGL in both tables)

In [ ]:
##Installing necessary packages

if (!require('tseries')) install.packages('tseries'); library(tseries)
if (!require('glasso')) install.packages('glasso'); library(glasso)
if (!require('huge')) install.packages('huge'); library(huge)
if (!require('pracma')) install.packages('pracma'); library(pracma)
if (!require('nlshrink')) install.packages('nlshrink'); library(nlshrink)
if (!require('glmnet')) install.packages('glmnet'); library(glmnet)
if (!require('missForest')) install.packages('missForest'); library(missForest)
if (!require('POET')) install.packages('POET'); library(POET)
if (!require('matrixcalc')) install.packages('matrixcalc'); library(matrixcalc)
if (!require('readr')) install.packages('readr'); library(readr)
if (!require('clime')) install.packages('clime'); library(clime)
if (!require('MASS')) install.packages('MASS'); library(MASS)
if (!require('devtools')) install.packages('devtools'); library(devtools)
devtools::install_github("cykbennie/fbi")
library(fbi)

In [ ]:
#######################################################################
##########FUNCTIONS FOR FORECAST COMBINATION###########
FindWeights=function(INV){
  ones=rep(1,nrow(INV))
  W=(INV%*%t(t(ones)))/(ones%*%INV%*%t(t(ones)))[[1]]
  return(W)
}

###updated DM test
dm_updated = function (e1, e2) {
  d <- c(abs(e1))^2 - c(abs(e2))^2
  statistic <- (length(d))^(0.5)*mean(d, na.rm = TRUE)/sd(d, na.rm = TRUE)
  pval <- 1 - pnorm(statistic, mean = 0, sd = 1)
  output <- list(cbind(d), c(statistic), c(pval))
  return(output)
}

# Nodewise estimation of the covariance matrix
est_ndwcov <- function(Y,ic){
  
  # initialization
  p <- ncol(Y)
  n <- nrow(Y)
  Y <- Y- t((apply(Y,2,mean))%*%matrix(1,1,n)) # Y is de-meaned
  
  C <- matrix(0,p,p)
  diag(C) <- 1
  tau <- NULL
  
  # Loop over the assets
  for(j in 1:p){
    # Estimate the Lasso
    jlas <- glmnet(x=Y[,-j],y=Y[,j],family = 'gaussian',intercept = FALSE)
    # Get fit
    jfit <- predict(jlas, newx=Y[,-j], type="response")    
    # residuals
    jres <- matrix(Y[,j],n,length(jlas$lambda)) - jfit
    # std err
    jsig <- colSums(jres^2)/n
    # Computing information criterion
    if(ic=='WIC') jbic  <- log(jsig) + jlas$df *log(n)/n * log(log(p)) # BIC (Wang,2010)
    if(ic=='GIC') jbic  <- log(jsig) + jlas$df *log(p)/n * log(log(n)) # GIC
    if(ic=='BIC') jbic  <- log(jsig) + jlas$df *log(n)/n  #BIC
    if(ic=='MIC') jbic  <- jsig + jlas$df *log(n) * log(log(p))/n # MC's IC
    if(ic=='AIC') jbic  <- log(jsig) + 2 * jlas$df # AIC
    # Index of selected model 
    jind  <- which.min(jbic)
    # Get the parameters
    jpar <- jlas$beta[,jind]
    # Computing tau squared
    jtau <- sum(jres[,jind]^2)/n + (1/2)*jlas$lambda[jind]*sum(abs(jpar)) # using (10)
    # Storing the parameters
    C[j,-j] <- -jpar
    tau <- c(tau,jtau)
  }
  
  # Construct T-squared inverse
  T2inv <- diag(1/tau)
  
  # Construct Theta-hat
  Theta <- T2inv %*% C
  
  # sparsity
  sp <- sum(Theta==0)/(p^2)
  
  return(list(NULL,Theta,sp))
}
###########################################
#######################################################################

### LOADING DATA INSTRUCTIONS:
#### ECB SPF:
- data for three series is located in DATA -> ECB SPF data -> navigate to the corresponding target series of interest when making selection
- "yhat.csv" contains forecasts, "forERR.csv has forecast errors, "ytrue.csv" has true series, "breaks.csv" contains a break vector for state breaks(as defined in the paper).

(Please refer to README.txt for more detailed data instructions)

In [ ]:
#######ECB SPF APPLICATION######
###Loading the data for ECB SPF application
#load the data that corresponds to your series of interest (navigate to DATA -> ECB SPF data folder and choose the target series folder)
yhat <- read_csv("yhat.csv")
forERR <- read_csv("forERR.csv")
ytrue <- read_csv("ytrue.csv")
Yhat <- as.matrix(yhat)
Yhat <- Yhat[1:82, ] 
# true <- as.matrix(Actual_gdpgrowth_data[,2])
# true = true/100
true <- as.matrix(ytrue[1:82, ]) 
forERR <- as.matrix(forERR)
forERR <- forERR[1:82, ] 
y_frac <- as.matrix(ytrue)

In [ ]:
##############################FORECAST COMBINATION ECB SPF################################

seqvec = seq(30,50,10)
MSFE_total <- matrix(0,(length(seqvec)),14)#14 is the number of competing methods 

for (l in 1:length(seqvec)){
R=seqvec[l]
# R=15
T=nrow(forERR)
m2 =T-R  #Forecasting observations
EW=as.matrix(rep(1/(ncol(forERR)),(ncol(forERR))))
####FORECAST ERRORS MATRICES####
FEEW= matrix(0,m2,1)
FEFGL= matrix(0,m2,1)
FEGL= matrix(0,m2,1)
FELW= matrix(0,m2,1)
FELW_set= matrix(0,m2,1)
FEPOET= matrix(0,m2,1)
FENLW= matrix(0,m2,1)
FENLW_set= matrix(0,m2,1)
FECLIME= matrix(0,m2,1)
FECLIME_set= matrix(0,m2,1)
FEnotsparse= matrix(0,m2,1)
FEMB= matrix(0,m2,1)
FEFMB= matrix(0,m2,1)
FEAR1= matrix(0,m2,1)
#####SFEs####
SFEEW= matrix(0,m2,1)
SFEFGL= matrix(0,m2,1)
SFEGL= matrix(0,m2,1)
SFELW= matrix(0,m2,1)
SFELW_set= matrix(0,m2,1)
SFEPOET= matrix(0,m2,1)
SFENLW= matrix(0,m2,1)
SFENLW_set= matrix(0,m2,1)
SFECLIME= matrix(0,m2,1)
SFECLIME_set= matrix(0,m2,1)
SFEnotsparse= matrix(0,m2,1)
SFEMB= matrix(0,m2,1)
SFEFMB= matrix(0,m2,1)
SFEAR1= matrix(0,m2,1)
for(j in 1:m2)  {
  print(paste0("l,j= ",l,",",j))
  set = forERR[j:(j+R-1),]
  # k=POETKhat(t(set))
  # k= k[["K1BN"]]
  # k=1
  k=2
  betas <- eigen(t(set)%*%set)[["vectors"]][,1:k]
  covariate <- as.matrix(prcomp(set)[["x"]][,(1:k)]) ##these are factors like Fhat
  betas = as.matrix(betas) # p*K
  betas = t(betas) #K*p
  residuals <- set - covariate%*%betas
  epshat <- as.matrix(residuals)
  
 #######FGL########
  covU = cov(epshat)
  fwgl <- rglasso(covU,weight = TRUE, N=nrow(set)-1)
  ThetaFWGL1 = fwgl[[2]]
  # est1 = huge(epshat, method = "glasso", lambda=NULL, scr = TRUE, cov.output = FALSE, verbose = FALSE)
  # out.select1 = huge.select(est1,criterion = "ebic",ebic.gamma = 1)
  # ThetaFWGL1 = out.select1[["opt.icov"]]
  if (k==1){
    ThetaFWGL2 = ThetaFWGL1 - ThetaFWGL1%*%t(betas)%*%solve( (var(covariate))^(-1)+
                                                               betas%*%ThetaFWGL1%*%t(betas))%*%betas%*%ThetaFWGL1
  } else {
    ThetaFWGL2 = ThetaFWGL1 - ThetaFWGL1%*%t(betas)%*%solve( solve(cov(covariate))+
                                                               betas%*%ThetaFWGL1%*%t(betas))%*%betas%*%ThetaFWGL1
  }
  
  ########GLASSO#######
  covE = cov(set)
  wgl <- rglasso(covE,weight = TRUE, N=nrow(set)-1)
  ThetaGL = wgl[[2]]
  # est2 = huge(set, method = "glasso", lambda=NULL, scr = TRUE, cov.output = FALSE, verbose = FALSE)
  # out.select2 = huge.select(est2,criterion = "ebic",ebic.gamma = 1)
  # ThetaGL = out.select2[["opt.icov"]]
  
  ########LW#############
  cov_LW = linshrink_cov(set, k = 0)
  
  LWTheta_set = solve(cov_LW)
  
  ########FLW#############
  covU_LW = linshrink_cov(epshat, k = 0)
  if (k==1){
    LWTheta = solve(covU_LW) -  solve(covU_LW)%*%t(betas)%*%solve( (var(covariate))^(-1)+
                                       betas%*% solve(covU_LW)%*%t(betas))%*%betas%*% solve(covU_LW)
  } else {
    LWTheta = solve(covU_LW) -  solve(covU_LW)%*%t(betas)%*%solve( solve(cov(covariate))+
                                       betas%*% solve(covU_LW)%*%t(betas))%*%betas%*% solve(covU_LW)
  }
  
  ########NLW#############
  cov_NL=nlshrink_cov(set, k = 0,method = "nloptr")
  NLWTheta_set = solve(cov_NL)
  
  ########FNLW#############
  covU_NL=nlshrink_cov(epshat, k = 0,method = "nloptr")
  if (is.singular.matrix(covU_NL)== TRUE){
    ThetaU_NL = ginv(covU_NL)
  } else {
    ThetaU_NL = solve(covU_NL)
  }
  if (k==1){
    NLWTheta = ThetaU_NL -  ThetaU_NL%*%t(betas)%*%solve( (var(covariate))^(-1)+
                          betas%*%ThetaU_NL%*%t(betas))%*%betas%*% ThetaU_NL
  } else {
    NLWTheta = ThetaU_NL -  ThetaU_NL%*%t(betas)%*%solve( solve(cov(covariate))+
                          betas%*% ThetaU_NL%*%t(betas))%*%betas%*% ThetaU_NL
  }
  ############POET##################
  
  covPOET=POET(t(epshat),K=k,0.5,"soft","vad")$SigmaY
  #covY are sample covariancematrix of returns by POET,0.5,SOFT,VAD ARE DEFAULTS, 
  #7 FACTORS ARE  RECOMMENDED BY POET PROGRAM,ALTERNATIVE IS ESTIMATE AND FIT
  ThetaPOET=solve(covPOET)#INVERSE MATRIX PRECISION ESTIMATE

  ##########CLIME########
  # clime_est = fastclime(set)
  # out2 = fastclime.selector(clime_est$lambdamtx, clime_est$icovlist,0.2)
  # ThetaFClime_set = out2$icov
  ##optimal lambda was giving an error of non-singularity, so just using lambda=0.8
  ## (which corresponded to smallest loss)
  ## FOR GDP set lambda=0.8
  ## FOR CPI set lambda=0.01 
  ##NOTE: sometimes linsolver="primaldual" raises non-singularity error -> can try simplex instead
  re.clime <- clime(set, standardize=FALSE, lambda = 0.01, linsolver="simplex") ##X is n (observations) times p (variables)
  # re.cv <- cv.clime(re.clime)
  # re.clime.opt <- clime(set, standardize=FALSE, re.cv$lambdaopt)
  ThetaFClime_set <- re.clime[["Omegalist"]][[1]]
  
  ##########FCLIME########
  ##same issue with using optimal lambda as above
  re.clime <- clime(epshat, standardize=FALSE, lambda = 0.01, linsolver="simplex") ##X is n (observations) times p (variables)
  # re.cv <- cv.clime(re.clime)
  # re.clime.opt <- clime(epshat, standardize=FALSE, re.cv$lambdaopt)
  # ClimeTheta <- re.clime.opt[["Omegalist"]][[1]]
  ClimeTheta <- re.clime[["Omegalist"]][[1]]
  
  
  # clime_est = fastclime(epshat)
  # out2 = fastclime.selector(clime_est$lambdamtx, clime_est$icovlist,0.2)
  # ClimeTheta = out2$icov
  if (k==1){
    ThetaFClime = ClimeTheta - ClimeTheta%*%t(betas)%*%solve( (var(covariate))^(-1)+
                                                                betas%*%ClimeTheta%*%t(betas))%*%betas%*%ClimeTheta
  } else {
    ThetaFClime = ClimeTheta - ClimeTheta%*%t(betas)%*%solve( solve(cov(covariate))+
                                                                betas%*%ClimeTheta%*%t(betas))%*%betas%*%ClimeTheta
  }

  ##########FFs WITHOUT SPARSITY RESTRICTION
  prec1 = ginv(cov(epshat))
  if (k==1){
  Theta1 = prec1 - prec1%*%t(betas)%*%solve( (var(covariate))^(-1)+
                                                betas%*%prec1%*%t(betas))%*%betas%*%prec1
  } else {
  Theta1 = prec1 - prec1%*%t(betas)%*%solve( solve(cov(covariate))+
                                                  betas%*%prec1%*%t(betas))%*%betas%*%prec1
  }
  ########################NODEWISE REGRESSION#########################
  ######################################################################
  nodewise <- est_ndwcov(set,ic = "GIC")
  ThetaMB <- nodewise[[2]]
  ########################FACTOR NODEWISE REGRESSION#########################
  ######################################################################
  nodewise_factor <- est_ndwcov(epshat,ic = "GIC")
  ThetaFMB_eps <- nodewise_factor[[2]]
  if (k==1){
    ThetaFMB = ThetaFMB_eps - ThetaFMB_eps%*%t(betas)%*%solve( (var(covariate))^(-1)+
                                                          betas%*%ThetaFMB_eps%*%t(betas))%*%betas%*%ThetaFMB_eps
  } else {
    ThetaFMB = ThetaFMB_eps - ThetaFMB_eps%*%t(betas)%*%solve( solve(cov(covariate))+
                                                          betas%*%ThetaFMB_eps%*%t(betas))%*%betas%*%ThetaFMB_eps
  }
  
  
  wGL=FindWeights(ThetaFWGL2)  
  wFGL=FindWeights(ThetaGL)
  wLW=FindWeights(LWTheta)
  wLW_set=FindWeights(LWTheta_set)
  wPOET=FindWeights(ThetaPOET)
  wNLW=FindWeights(NLWTheta)
  wNLW_set=FindWeights(NLWTheta_set)
  wCLIME=FindWeights(ThetaFClime)
  wCLIME_set=FindWeights(ThetaFClime_set)
  wnotsparse=FindWeights(Theta1)
  wMB=FindWeights(ThetaMB)
  wFMB=FindWeights(ThetaFMB)
  
###############################################
##############AR(1) FORECAST#################
myts <- ts(ytrue_ts[,2], start=c(1999,4), end=c(2024, 1), frequency=4)  
  
ytrue_ar <- myts[j:(j+R-1)]  
ar1 <- arima(ytrue_ar, order = c(1,0,0), method="CSS") # method="ML" forces the model to be stationary
#BUT it sometimes raises an error that the system is singular, so can choose method="CSS" 
pred_ar = y_frac[R]*ar1[["coef"]][[1]]  + ar1[["coef"]][[2]]
FEAR1[j] = y_frac[R +j] - pred_ar
###############MODEL FORECASTS ARE HERE######
FEFGL[j]=y_frac[R +j]- t(wFGL)%*%Yhat[R +j,]
FEEW[j]=y_frac[R +j]- t(EW)%*%Yhat[R +j,]
FEGL[j]= y_frac[R +j]- t(wGL)%*%Yhat[R +j,]
FELW[j]=y_frac[R +j]- t(wLW)%*%Yhat[R +j,]
FELW_set[j]=y_frac[R +j]- t(wLW_set)%*%Yhat[R +j,]
FEPOET[j]= y_frac[R +j]- t(wPOET)%*%Yhat[R +j,]
FENLW[j]= y_frac[R +j]- t(wNLW)%*%Yhat[R +j,]
FENLW_set[j]= y_frac[R +j]- t(wNLW_set)%*%Yhat[R +j,]
FECLIME[j]= y_frac[R +j]- t(wCLIME)%*%Yhat[R +j,]
FECLIME_set[j]= y_frac[R +j]- t(wCLIME_set)%*%Yhat[R +j,]
FEnotsparse[j]= y_frac[R +j]- t(wnotsparse)%*%Yhat[R +j,]
FEMB[j]=y_frac[R +j]- t(wMB)%*%Yhat[R +j,]
FEFMB[j]= y_frac[R +j]- t(wFMB)%*%Yhat[R +j,]

#################SFEs ARE HERE##################
  SFEAR1[j] = (FEAR1[j])^2
  SFEFGL[j]=(y_frac[R +j]- t(wFGL)%*%Yhat[R +j,])^(2)
  SFEEW[j]=(y_frac[R +j]- t(EW)%*%Yhat[R +j,])^(2)
  SFEGL[j]=(y_frac[R +j]- t(wGL)%*%Yhat[R +j,])^(2)
  SFELW[j]=(y_frac[R +j]- t(wLW)%*%Yhat[R +j,])^(2)
  SFELW_set[j]=(y_frac[R +j]- t(wLW_set)%*%Yhat[R +j,])^(2)
  SFEPOET[j]=(y_frac[R +j]- t(wPOET)%*%Yhat[R +j,])^(2)
  SFENLW[j]=(y_frac[R +j]- t(wNLW)%*%Yhat[R +j,])^(2)
  SFENLW_set[j]=(y_frac[R +j]- t(wNLW_set)%*%Yhat[R +j,])^(2)
  SFECLIME[j]=(y_frac[R +j]- t(wCLIME)%*%Yhat[R +j,])^(2)
  SFECLIME_set[j]=(y_frac[R +j]- t(wCLIME_set)%*%Yhat[R +j,])^(2)
  SFEnotsparse[j]=(y_frac[R +j]- t(wnotsparse)%*%Yhat[R +j,])^(2)
  SFEMB[j]=(y_frac[R +j]- t(wMB)%*%Yhat[R +j,])^(2)
  SFEFMB[j]=(y_frac[R +j]- t(wFMB)%*%Yhat[R +j,])^(2)
  

##################
  
}
MSFE1=mean(SFEEW)
MSFE2=mean(SFEFGL)
MSFE3=mean(SFEGL)
MSFE4=mean(SFELW)
MSFE5=mean(SFENLW)
MSFE6=mean(SFEPOET)
MSFE7=mean(SFECLIME)
MSFE8=mean(SFEnotsparse)
MSFE9=mean(SFEMB)
MSFE10=mean(SFEFMB)
MSFE11=mean(SFELW_set)
MSFE12=mean(SFENLW_set)
MSFE13=mean(SFECLIME_set)
MSFE14=mean(SFEAR1)


A = NULL
A = rbind(A,c(MSFE1, MSFE2, MSFE3, MSFE4, MSFE5, MSFE6,MSFE7, MSFE8,MSFE9,MSFE10,MSFE11,MSFE12,MSFE13, MSFE14))
write.csv(A,file="MSFE_ECB.csv")

### LOADING DATA INSTRUCTIONS:
#### FRED MD:
- data for four series is located in DATA -> FRED MD data -> navigate to the corresponding target series of interest when making selection
- "yhat.csv" contains forecasts, "forERR.csv has forecast errors, "ytrue.csv" has true series, "breaks.csv" contains a break vector for state breaks(as defined in the paper).
- In case you would like to update the data, we followed the following steps to obtain forecasts:
 1) use the code cell titled "Updating FRED MD forecasts" below to obtain forecasts up to 4 steps ahead

(Please refer to README.txt for more detailed data instructions)

In [ ]:
##UPDATING FRED MD forecasts (DON'T RUN UNLESS you would like to reproduce forecasts OR update forecasts using recent data)

data = fredmd(file='current.csv', transform = TRUE)

write.csv(data,file="data.csv")

X1 <- as.matrix(data)
X1 <- X1[,-c(24)] ## 6 is INDPROD, 105 is CPIAUCSL, 115 is PCEPI, 24 is UNRATE
X1 <- X1[(1+h):788,]
y1 <- as.matrix(current)
y <- y[2:789]

scaled.X1 <- scale(X1, center = TRUE, scale = TRUE)
IMPscaled.X1 <- missForest(scaled.X1)
scaled.X1 <- as.matrix(IMPscaled.X1[["ximp"]])
X <- scaled.X1
y <- y1[2:788,24:24] 

sum(is.na(y))
sum(is.na(X))
y_backup <- y
X_backup <- X

#set forecasting horizon
h=1
###FOR GROWTH RATES###
y_g <- NULL
count=0
for (i in 2:(length(y_backup))) {
  count = count + 1
  y_g[count] = (1/h)*log(y_backup[i]/y_backup[i-h])
}

y <- y_g

###FOR CHANGES### (E.G. UNRATE)
y_g <- NULL
count=0
for (i in 2:(length(y_backup))) {
  count = count + 1
  y_g[count] = (1/h)*(y_backup[i]-y_backup[i-h])
}

y <- y_g


##DATA for FRED-MD Application can be found in DATA folder (X is the dataset of macro series excluding the target,
#and y is the target series (INDPROD, CPIAUCSL, PCEPI, or UNRATE))

#1) First we need to produce forecasts (Yhats) and forecast errors for FRED-MD application, 
#which is done by forecasting the target with each column of X one at a time

###forecasting with each column at a time
T=nrow(X)
m1=240
m2 =T-m1  #Forecasting observations

####for trying if the code works for smaller sample size

Yhat = matrix(0,m2,ncol(X))
Yhat2 = matrix(0,(m2-2+1),ncol(X))
Yhat3 = matrix(0,(m2-3+1),ncol(X))
Yhat4 = matrix(0,(m2-4+1),ncol(X))


for (col in 1:ncol(X))  {
  x1 = X[,col]
  for (row in 1:m2){
    ######################
    #FOR h=1
    ######################
    const = ones((m1-1),1)
    regr <- x1[1:(m1-1)]
    xx = cbind(const,regr)
    
    coef = solve(t(xx)%*%xx)%*%t(xx)%*%y[(row+1):(m1-1+row)]
    
    Yhat[row,col] = c(1, x1[m1-1+row])%*%coef
    
    # model = lm(y[(row+h):(m1-1+row)] ~ regr) #checking if using LM gives the same result (it does)
  }
    ######################
    #FOR h=2
    ######################
  for (row2 in 1:(m2-2+1)){
    const = ones((m1-2),1)
    regr <- x1[1:(m1-2)]
    xx = cbind(const,regr)
    
    coef = solve(t(xx)%*%xx)%*%t(xx)%*%y[(row2+2):(m1-1+row2)]
    
    Yhat2[row2,col] = c(1, x1[m1-1+row2])%*%coef
  }
    ######################
    #FOR h=3
    ######################
  for (row3 in 1:(m2-3+1)){
    const = ones((m1-3),1)
    regr <- x1[1:(m1-3)]
    xx = cbind(const,regr)
    
    coef = solve(t(xx)%*%xx)%*%t(xx)%*%y[(row3+3):(m1-1+row3)]
    
    Yhat3[row3,col] = c(1, x1[m1-1+row3])%*%coef
  }
    ######################
    #FOR h=4
    ######################
  for (row4 in 1:(m2-4+1)){
    const = ones((m1-4),1)
    regr <- x1[1:(m1-4)]
    xx = cbind(const,regr)
    
    coef = solve(t(xx)%*%xx)%*%t(xx)%*%y[(row4+4):(m1-1+row4)]
    
    Yhat4[row4,col] = c(1, x1[m1-1+row4])%*%coef

  }
}

#####computing forecast errors
forERR=matrix(y[(m1 +1):T], m2, (ncol(X)))-Yhat
forERR2=matrix(y[(m1 +2):T], (m2-2+1), ncol(X))-Yhat2
forERR3=matrix(y[(m1 +3):T], (m2-3+1), ncol(X))-Yhat3
forERR4=matrix(y[(m1 +4):T], (m2-4+1), ncol(X))-Yhat4


write.csv(Yhat,file="yhat.csv")
write.csv(forERR,file="forERR.csv")

write.csv(Yhat2,file="yhat2.csv")
write.csv(forERR2,file="forERR2.csv")

write.csv(Yhat3,file="yhat3.csv")
write.csv(forERR3,file="forERR3.csv")

write.csv(Yhat4,file="yhat4.csv")
write.csv(forERR4,file="forERR4.csv")

In [ ]:
##############################FORECAST COMBINATION FRED-MD################################
## DATA -> FRED MD data -> navigate to the corresponding target series of interest
yhat <- read_csv("yhat.csv", col_types = cols(...1 = col_skip()))
forERR <- read_csv("forERR.csv", col_types = cols(...1 = col_skip()))
true <- read_csv("true.csv", col_types = cols(...1 = col_skip()))
Yhat <- as.matrix(yhat)
true <- as.matrix(true)
forERR <- as.matrix(forERR)
y_frac <- true

#2) Once we have the forecasts we can proceed to forecast combination
seqvec = seq(400,400,100)
MSFE_total <- matrix(0,(length(seqvec)),14)#14 is the number of competing methods
PVAL_total <- matrix(0,(length(seqvec)),14) #13 is the #of competing methods (MSFE_TOTAL-1 + EW as first element of 0 since its a threshold)
statGL <- list()
statFGL <- list()
statFLW <- list()
statLW <- list()
statPOET <- list()
statFNLW <- list()
statNLW <- list()
statFCLIME <- list()
statCLIME <- list()
statnotsparse <- list()
statMB <- list()
statFMB <- list()
statAR <- list()
for (l in 1:length(seqvec)){
  R=seqvec[l]
  T=nrow(forERR)
  m2 =T-R  #Forecasting observations
  EW=as.matrix(rep(1/(ncol(forERR)),(ncol(forERR))))
  ####FORECAST ERRORS MATRICES####
  FEEW= matrix(0,m2,1)
  FEFGL= matrix(0,m2,1)
  FEGL= matrix(0,m2,1)
  FELW= matrix(0,m2,1)
  FELW_set= matrix(0,m2,1)
  FEPOET= matrix(0,m2,1)
  FENLW= matrix(0,m2,1)
  FENLW_set= matrix(0,m2,1)
  FECLIME= matrix(0,m2,1)
  FECLIME_set= matrix(0,m2,1)
  FEnotsparse= matrix(0,m2,1)
  FEMB= matrix(0,m2,1)
  FEFMB= matrix(0,m2,1)
  FEAR1= matrix(0,m2,1)
  #####SFEs####
  SFEEW= matrix(0,m2,1)
  SFEFGL= matrix(0,m2,1)
  SFEGL= matrix(0,m2,1)
  SFELW= matrix(0,m2,1)
  SFELW_set= matrix(0,m2,1)
  SFEPOET= matrix(0,m2,1)
  SFENLW= matrix(0,m2,1)
  SFENLW_set= matrix(0,m2,1)
  SFECLIME= matrix(0,m2,1)
  SFECLIME_set= matrix(0,m2,1)
  SFEnotsparse= matrix(0,m2,1)
  SFEMB= matrix(0,m2,1)
  SFEFMB= matrix(0,m2,1)
  SFEAR1= matrix(0,m2,1)
  for(j in 1:m2)  {
    print(paste0("l,j= ",l,",",j))
    set = forERR[j:(j+R-1),]
    # k=POETKhat(t(set))
    # k= k[["K1BN"]]
    # k=1
    k=2
    betas <- eigen(t(set)%*%set)[["vectors"]][,1:k]
    covariate <- as.matrix(prcomp(set)[["x"]][,(1:k)]) ##these are factors like Fhat
    betas = as.matrix(betas) # p*K
    betas = t(betas) #K*p
    residuals <- set - covariate%*%betas
    epshat <- as.matrix(residuals)
    
    #######FGL########
    covU = cov(epshat)
    # fwgl <- rglasso(covU,weight = TRUE, N=nrow(set)-1)
    # ThetaFWGL1 = fwgl[[2]]
    est1 = huge(epshat, method = "glasso", lambda=NULL, scr = TRUE, cov.output = FALSE, verbose = FALSE)
    out.select1 = huge.select(est1,criterion = "ebic",ebic.gamma = 1)
    ThetaFWGL1 = out.select1[["opt.icov"]]
    if (k==1){
      ThetaFWGL2 = ThetaFWGL1 - ThetaFWGL1%*%t(betas)%*%solve( (var(covariate))^(-1)+
                                                                 betas%*%ThetaFWGL1%*%t(betas))%*%betas%*%ThetaFWGL1
    } else {
      ThetaFWGL2 = ThetaFWGL1 - ThetaFWGL1%*%t(betas)%*%solve( solve(cov(covariate))+
                                                                 betas%*%ThetaFWGL1%*%t(betas))%*%betas%*%ThetaFWGL1
    }
    
    ########GLASSO#######
    covE = cov(set)
    # wgl <- rglasso(covE,weight = TRUE, N=nrow(set)-1)
    # ThetaGL = wgl[[2]]
    # set_scaled <- scale(set)
    est2 = huge(set, method = "glasso", lambda=NULL, scr = TRUE, cov.output = FALSE, verbose = FALSE)
    out.select2 = huge.select(est2,criterion = "ebic",ebic.gamma = 1)
    ThetaGL = out.select2[["opt.icov"]]
    
    ########LW#############
    cov_LW = linshrink_cov(set, k = 0)
    
    LWTheta_set = solve(cov_LW)
    
    ########FLW#############
    covU_LW = linshrink_cov(epshat, k = 0)
    if (k==1){
      LWTheta = solve(covU_LW) -  solve(covU_LW)%*%t(betas)%*%solve( (var(covariate))^(-1)+
                                                                       betas%*% solve(covU_LW)%*%t(betas))%*%betas%*% solve(covU_LW)
    } else {
      LWTheta = solve(covU_LW) -  solve(covU_LW)%*%t(betas)%*%solve( solve(cov(covariate))+
                                                                       betas%*% solve(covU_LW)%*%t(betas))%*%betas%*% solve(covU_LW)
    }
    
    ########NLW#############
    cov_NL=nlshrink_cov(set, k = 0,method = "nloptr")
    NLWTheta_set = solve(cov_NL)
    
    ########FNLW#############
    covU_NL=nlshrink_cov(epshat, k = 0,method = "nloptr")
    if (is.singular.matrix(covU_NL)== TRUE){
      ThetaU_NL = ginv(covU_NL)
    } else {
      ThetaU_NL = solve(covU_NL)
    }
    if (k==1){
      NLWTheta = ThetaU_NL -  ThetaU_NL%*%t(betas)%*%solve( (var(covariate))^(-1)+
                                                              betas%*%ThetaU_NL%*%t(betas))%*%betas%*% ThetaU_NL
    } else {
      NLWTheta = ThetaU_NL -  ThetaU_NL%*%t(betas)%*%solve( solve(cov(covariate))+
                                                              betas%*% ThetaU_NL%*%t(betas))%*%betas%*% ThetaU_NL
    }
    ############POET##################
    
    covPOET=POET(t(epshat),K=k,0.5,"soft","vad")$SigmaY
    #covY are sample covariancematrix of returns by POET,0.5,SOFT,VAD ARE DEFAULTS, 
    ThetaPOET=solve(covPOET)#INVERSE MATRIX PRECISION ESTIMATE
    
    ##########CLIME########
    # clime_est = fastclime(set)
    # out2 = fastclime.selector(clime_est$lambdamtx, clime_est$icovlist,0.2)
    # ThetaFClime_set = out2$icov
    ##optimal lambda was giving an error of non-singularity, so just using lambda=0.8
    ## (which corresponded to smallest loss)
    ## FOR GDP set lambda=0.8
    ## FOR CPI set lambda=0.01 
    ##NOTE: sometimes linsolver="primaldual" raises non-singularity error -> can try simplex instead
    re.clime <- clime(set, standardize=FALSE, lambda = 0.01, linsolver="simplex") ##X is n (observations) times p (variables)
    # re.cv <- cv.clime(re.clime)
    # re.clime.opt <- clime(set, standardize=FALSE, re.cv$lambdaopt)
    ThetaFClime_set <- re.clime[["Omegalist"]][[1]]
    
    ##########FCLIME########
    ##same issue with using optimal lambda as above
    re.clime <- clime(epshat, standardize=FALSE, lambda = 0.01, linsolver="simplex") ##X is n (observations) times p (variables)
    # re.cv <- cv.clime(re.clime)
    # re.clime.opt <- clime(epshat, standardize=FALSE, re.cv$lambdaopt)
    # ClimeTheta <- re.clime.opt[["Omegalist"]][[1]]
    ClimeTheta <- re.clime[["Omegalist"]][[1]]
    
    
    # clime_est = fastclime(epshat)
    # out2 = fastclime.selector(clime_est$lambdamtx, clime_est$icovlist,0.2)
    # ClimeTheta = out2$icov
    if (k==1){
      ThetaFClime = ClimeTheta - ClimeTheta%*%t(betas)%*%solve( (var(covariate))^(-1)+
                                                                  betas%*%ClimeTheta%*%t(betas))%*%betas%*%ClimeTheta
    } else {
      ThetaFClime = ClimeTheta - ClimeTheta%*%t(betas)%*%solve( solve(cov(covariate))+
                                                                  betas%*%ClimeTheta%*%t(betas))%*%betas%*%ClimeTheta
    }
    
    ##########FFs WITHOUT SPARSITY RESTRICTION
    prec1 = ginv(cov(epshat))
    if (k==1){
      Theta1 = prec1 - prec1%*%t(betas)%*%solve( (var(covariate))^(-1)+
                                                   betas%*%prec1%*%t(betas))%*%betas%*%prec1
    } else {
      Theta1 = prec1 - prec1%*%t(betas)%*%solve( solve(cov(covariate))+
                                                   betas%*%prec1%*%t(betas))%*%betas%*%prec1
    }
    ########################NODEWISE REGRESSION#########################
    ######################################################################
    nodewise <- est_ndwcov(set,ic = "GIC")
    ThetaMB <- nodewise[[2]]
    ########################FACTOR NODEWISE REGRESSION#########################
    ######################################################################
    nodewise_factor <- est_ndwcov(epshat,ic = "GIC")
    ThetaFMB_eps <- nodewise_factor[[2]]
    if (k==1){
      ThetaFMB = ThetaFMB_eps - ThetaFMB_eps%*%t(betas)%*%solve( (var(covariate))^(-1)+
                                                                   betas%*%ThetaFMB_eps%*%t(betas))%*%betas%*%ThetaFMB_eps
    } else {
      ThetaFMB = ThetaFMB_eps - ThetaFMB_eps%*%t(betas)%*%solve( solve(cov(covariate))+
                                                                   betas%*%ThetaFMB_eps%*%t(betas))%*%betas%*%ThetaFMB_eps
    }
    
    wGL=FindWeights(ThetaFWGL2)
    wFGL=FindWeights(ThetaGL) 
    wLW=FindWeights(LWTheta)
    wLW_set=FindWeights(LWTheta_set)
    wPOET=FindWeights(ThetaPOET)
    wNLW=FindWeights(NLWTheta)
    wNLW_set=FindWeights(NLWTheta_set)
    wCLIME=FindWeights(ThetaFClime)
    wCLIME_set=FindWeights(ThetaFClime_set)
    wnotsparse=FindWeights(Theta1)
    wMB=FindWeights(ThetaMB)
    wFMB=FindWeights(ThetaFMB)
    
    ###############################################
    ##############AR(1) FORECAST#################
    myts <- ts(ytrue_ts[,2], start=c(1959,2,1), end=c(2024, 8,1), frequency=12) 
    
    ytrue_ar <- myts[j:(j+R-1)]  
    ar1 <- arima(ytrue_ar, order = c(1,0,0), method="CSS") # method="ML" forces the model to be stationary
    #BUT it sometimes raises an error that the system is singular, so can choose method="CSS" 
    pred_ar = y_frac[R]*ar1[["coef"]][[1]]  + ar1[["coef"]][[2]]
    FEAR1[j] = y_frac[R +j] - pred_ar
    ###############MODEL FORECASTS ARE HERE######
    FEFGL[j]=y_frac[R +j]- t(wFGL)%*%Yhat[R +j,]
    FEEW[j]=y_frac[R +j]- t(EW)%*%Yhat[R +j,]
    FEGL[j]= y_frac[R +j]- t(wGL)%*%Yhat[R +j,]
    FELW[j]=y_frac[R +j]- t(wLW)%*%Yhat[R +j,]
    FELW_set[j]=y_frac[R +j]- t(wLW_set)%*%Yhat[R +j,]
    FEPOET[j]= y_frac[R +j]- t(wPOET)%*%Yhat[R +j,]
    FENLW[j]= y_frac[R +j]- t(wNLW)%*%Yhat[R +j,]
    FENLW_set[j]= y_frac[R +j]- t(wNLW_set)%*%Yhat[R +j,]
    FECLIME[j]= y_frac[R +j]- t(wCLIME)%*%Yhat[R +j,]
    FECLIME_set[j]= y_frac[R +j]- t(wCLIME_set)%*%Yhat[R +j,]
    FEnotsparse[j]= y_frac[R +j]- t(wnotsparse)%*%Yhat[R +j,]
    FEMB[j]=y_frac[R +j]- t(wMB)%*%Yhat[R +j,]
    FEFMB[j]= y_frac[R +j]- t(wFMB)%*%Yhat[R +j,]
    
    #################SFEs ARE HERE##################
    SFEAR1[j] = (FEAR1[j])^2
    SFEFGL[j]=(y_frac[R +j]- t(wFGL)%*%Yhat[R +j,])^(2)
    SFEEW[j]=(y_frac[R +j]- t(EW)%*%Yhat[R +j,])^(2)
    SFEGL[j]=(y_frac[R +j]- t(wGL)%*%Yhat[R +j,])^(2)
    SFELW[j]=(y_frac[R +j]- t(wLW)%*%Yhat[R +j,])^(2)
    SFELW_set[j]=(y_frac[R +j]- t(wLW_set)%*%Yhat[R +j,])^(2)
    SFEPOET[j]=(y_frac[R +j]- t(wPOET)%*%Yhat[R +j,])^(2)
    SFENLW[j]=(y_frac[R +j]- t(wNLW)%*%Yhat[R +j,])^(2)
    SFENLW_set[j]=(y_frac[R +j]- t(wNLW_set)%*%Yhat[R +j,])^(2)
    SFECLIME[j]=(y_frac[R +j]- t(wCLIME)%*%Yhat[R +j,])^(2)
    SFECLIME_set[j]=(y_frac[R +j]- t(wCLIME_set)%*%Yhat[R +j,])^(2)
    SFEnotsparse[j]=(y_frac[R +j]- t(wnotsparse)%*%Yhat[R +j,])^(2)
    SFEMB[j]=(y_frac[R +j]- t(wMB)%*%Yhat[R +j,])^(2)
    SFEFMB[j]=(y_frac[R +j]- t(wFMB)%*%Yhat[R +j,])^(2)
    
    
    ##################
    
  }
  MSFE1=mean(SFEEW)
  MSFE2=mean(SFEFGL)
  MSFE3=mean(SFEGL)
  MSFE4=mean(SFELW)
  MSFE5=mean(SFENLW)
  MSFE6=mean(SFEPOET)
  MSFE7=mean(SFECLIME)
  MSFE8=mean(SFEnotsparse)
  MSFE9=mean(SFEMB)
  MSFE10=mean(SFEFMB)
  MSFE11=mean(SFELW_set)
  MSFE12=mean(SFENLW_set)
  MSFE13=mean(SFECLIME_set)
  MSFE14=mean(SFEAR1)
  
  
  A = NULL
  A = rbind(A,c(MSFE1, MSFE2, MSFE3, MSFE4, MSFE5, MSFE6,MSFE7, MSFE8,MSFE9,MSFE10,MSFE11,MSFE12,MSFE13, MSFE14))
  write.csv(A,file="MSFE_FRED.csv")